In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# 各パス指定

In [40]:
dataset = r'C:\Users\poooo\OneDrive\Desktop\grad_proj\Final\hand-gesture-recognition-mediapipe-main\captured_signs.csv'
model_save_path = 'model\keypoint_classifier\keypoint_classifier.keras'

<>:2: SyntaxWarning: invalid escape sequence '\k'
<>:2: SyntaxWarning: invalid escape sequence '\k'
C:\Users\poooo\AppData\Local\Temp\ipykernel_104676\2560058486.py:2: SyntaxWarning: invalid escape sequence '\k'
  model_save_path = 'model\keypoint_classifier\keypoint_classifier.keras'


# 分類数設定

In [42]:
NUM_CLASSES = 2
print(NUM_CLASSES)

2


# 学習データ読み込み

In [43]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [44]:
X_dataset

array([[ 0.        ,  0.        ,  0.18666667, ..., -0.3511111 ,
        -0.15111111, -0.29333332],
       [ 0.        ,  0.        ,  0.17030568, ..., -0.34934497,
        -0.16157205, -0.29257643],
       [ 0.        ,  0.        ,  0.17167382, ..., -0.36480686,
        -0.16309012, -0.3090129 ],
       ...,
       [ 0.        ,  0.        , -0.08510638, ..., -0.34893617,
         0.2212766 , -0.28936172],
       [ 0.        ,  0.        , -0.08974359, ..., -0.35897437,
         0.21794872, -0.2948718 ],
       [ 0.        ,  0.        , -0.08474576, ..., -0.3516949 ,
         0.22033899, -0.2881356 ]], dtype=float32)

In [45]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [46]:
y_dataset

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [48]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [49]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,092 (4.27 KB)

 Trainable params: 1,092 (4.27 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [50]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# モデル訓練

In [51]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    # callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6765 - loss: 0.6783 - val_accuracy: 0.6667 - val_loss: 0.6654
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.6765 - loss: 0.7671 - val_accuracy: 0.6667 - val_loss: 0.6630
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.6765 - loss: 0.6615 - val_accuracy: 0.6667 - val_loss: 0.6611
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.6765 - loss: 0.6578 - val_accuracy: 0.6667 - val_loss: 0.6593
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.6765 - loss: 0.7017 - val_accuracy: 0.6667 - val_loss: 0.6572
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.6765 - loss: 0.7590 - val_accuracy: 0.6667 - val_loss: 0.6555
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.6765 - loss: 0.6462 - val_accuracy: 0.6667 - val_loss: 0.6540
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.6765 - loss: 0.7476 - val_accuracy: 0.666

In [52]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 1.0000 - loss: 0.0155


In [53]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [54]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [55]:
# 推論テスト
predict_result = model.predict(np.array([X_test[4]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[0.00368533 0.9963147 ]
1


# 混同行列

# Tensorflow-Lite用のモデルへ変換

In [17]:
# モデルを変換(量子化)
# tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quantized_model = converter.convert()

# open(tflite_save_path, 'wb').write(tflite_quantized_model)

# 推論テスト

In [18]:
# interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
# interpreter.allocate_tensors()

In [19]:
# # 入出力テンソルを取得
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

In [20]:
# interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [21]:
# %%time
# # 推論実施
# interpreter.invoke()
# tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [22]:
# print(np.squeeze(tflite_results))
# print(np.argmax(np.squeeze(tflite_results)))